![Callysto.ca Banner](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-top.jpg?raw=true)



## plants-watering

This is a short notebook to collect and display data from "Phidgets" sensors around a plant and allow the user to water the plant.

We have tried to make this notebook as friendly as possible for beginner programmers and hardware hackers.

Ask M. Lamoureux or Mary Grant (Callysto ambassadors) for details. August, 2023.

<h2 style="color: red">IMPORTANT</h2>
You MUST click the <span style="color:red">BIG, RED DISCONNECT BUTTON</span> at the end of this notebook when you are done. This tells the software to release the Phidget device, so it can be used by other notebooks you might try later.

## Overview

The purpose of this notebook is to collect data from certain hardware sensors called "Phidgets" which are set up to monitor the environment of a plant, and also to water the plant. The data can be viewed directly on the computer screen, and a water pump can be activated from a simple water pump.

We are using a complete kit for plant monitoring that is available from the Phidgets company (https://phidgets.com) This kit has all the sensors we need to monitor the environment of the plant, along with a water pump and control to water the plant. 

We recommend you follow the detailed setup for the Plant Kit provided by the Phidget company. The instructions are here: https://www.phidgets.com/education/learn/projects/plant-kit/introduction/

The Phidget Plant Kit contains several components. To set up this notebook, make sure you have the following:

- the Phidget VINT Hub (HUB0000), with cables attached
- the DC power supply (PSU2000), attached to Port 1 on the VINT Hub
- the water pump (KIT4014), attached to the power supply, 
- the moisture Phidget (device type HUM1100), attached to the VINT Hub
- the light sensor (device type LUX1000_0), attached to the VINT Hub
- and optionally, the temperature/humidity sensor (device type HUM1001_01).

There are for main steps in this notebook.

1. Set up gauges on the screen to view the data from the plant sensors.
2. Set up the plant sensors as software inputs to computer. Ensure that the softare can communicate with them.
3. Test that the water pump works by pressing the on-screen button.
5. Observe the changes in the moisture readings once the plant has been watered. 

Run the cells in this notebook, one cell at a time. This will give you the chance to respond to any errors and fix things if necessary. Do not select "Run All." 

You should have a plant, with the sensors, water reservoir, and pump  nearby. This what the plant setup should look like. 

**Note: Keep it away from computer and electronic devices as much as possible.**

<div align="center">
<img src="images/plant-water.jpg" alt="A plant with sensors and water pump" width="400"/><br>
A plant with sensors and water pump.
</div>

Here is some more detail on the various devices we are using. 

<div align="center">
<img src="images/sensors.jpg" alt="Details of the pump and sensors" width="600"/><br>
1: Pump. 2: DC power supply. 3: Moisture probe. 4: Light sensor. 5: Humidity sensor. 6: VINT Hub
</div>

The notebook runs both Python code and Javascript (JS) code. It would be nice to do everything in Python, but it seems Javascript is necessary to communicate with the Phidgets in a Jupyter notebook. Fortunately, the Javascript code here is easy enough to read and you will not need to change it. 

## Step 1. Setting up the gauges

We first set up some gauges to display values for temperature, humidity, soil moisture and light levels. This uses the Plotly library, which is loaded in with the **import** command in Python. 

We also include a button that will be used to water the plant. 

The gauges are not yet connected to the sensors, so they just display some default values.

In [ ]:
from IPython.display import display, Javascript
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import ipywidgets as widgets
from time import sleep

# the four sensor gauges
g_temp = go.Indicator(
    mode = "gauge+number",
    value = 20,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Temperature"},
    gauge = {'axis': {'range': [10, 40]}}
)

g_hum = go.Indicator(
    mode = "gauge+number",
    value = 40,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Humidity"},
    gauge = {'axis': {'range': [0, 100]}}
)

g_moist = go.Indicator(
    mode = "gauge+number",
    value = 0.5,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Moisture"},
    gauge = {'axis': {'range': [0, 1.0]}}
)

g_light = go.Indicator(
    mode = "gauge+number",
    value = 40,
    domain = {'x': [0, 1], 'y': [0, 1]},
    title = {'text': "Light Level"},
    gauge = {'axis': {'range': [0, 10000]}}
)

fig = make_subplots(
    rows=2,
    cols=2,
    specs=[[{'type' : 'domain'}, {'type' : 'domain'}],[{'type' : 'domain'}, {'type' : 'domain'}]],
    vertical_spacing = 0.35
)
fig.append_trace(g_temp, row=1, col=1)
fig.append_trace(g_hum, row=1, col=2)
fig.append_trace(g_moist, row=2, col=1)
fig.append_trace(g_light, row=2, col=2)

gauges = go.FigureWidget(fig)

# the data update button
data_button = widgets.Button(
    description='Update gauges',
    tooltip='Update data to gauges',
    disabled=False,
    button_style='success',
)

# the watering button
water_button = widgets.Button(
    description='Click to water',
    disabled=False,
    button_style='success',
    tooltip='Water for 2 seconds',
)
    
dashboard = widgets.VBox([gauges,widgets.HBox([data_button,water_button])])

display(dashboard)


## Step 2: Connecting the sensor devices

At this point, you should connect the Phidgets hardware to your computer. This includes six separate devices:
- the Phidget VINT device, attached to the computer's USB port
- the DC power supply, attached to Port 1 on the VINT device
- The water pump, attached to the DC power supply
- the soil moisture sensor, attached to the VINT
- the light sensor, attached to the VINT
- optionally, the combined temperature/humidity sensor, attached to the VINT

When you first attach the VINT device to the USB port, your computer may ask you whether you wish to connect to this USB device. Please answer "yes" to this security request. 

We now go set up the software to communicate with the Phidget devices.

### Load the phidget library for Javascript

In [ ]:
%%js
requirejs.config({
    paths: { 
        'phidget22': ['https://unpkg.com/phidget22/browser/phidget22'], 
    },                                         
});
require(['phidget22'], (phidget22) => {
   window.phidget22 = phidget22; 
});

In [ ]:
## We pause for a second here, to allow some time for the library to load in the background
sleep(1)

### Opening the USB connection

We now open a connection between the computer and your Phidget VINT device. The VINT must be plugged into your computer's USB port. Run the following cell, and follow the prompts to select the VINT device (a list appears that you should click on). This will pair the device with your computer. 

In [ ]:
%%js
if (window.usbconn === undefined) {
    element.text("Creating a new USB Connection.");
    window.usbconn = new phidget22.USBConnection();    
    usbconn.connect().then(() => {
        usbconn.requestWebUSBDeviceAccess();
    }).catch(err => {
        window.usbconn.delete();
        element.append("Error connecting to USB" + err);
    });
}

In [ ]:
## We rest for a bit while the USB connects
sleep(1)

### Confirm the USB connection 

Run the following code to see if the device is connected. It should say "true." If it does not, check your cable connections. You may also need to check the security settings on your computer to allow new USB devices to get connected. 

In [ ]:
%%js
element.text("Is the USB device connected? " + (usbconn.connected ? "YES.":"NO."));

### Connect the sensors

We make a request to open the four different sensors, for temperature, humidity, soil moisture and light level, as well as the control for the water pump. Be sure your devi es sensors are plugged into the VINT device. 

If you are missing a sensor or two, that is okay. The data collection for the other sensors will still work. The cells below will connect the sensors, then check to see that they are attached. 

In [ ]:
%%js
// minimize the async parts

window.tempSensor = new phidget22.TemperatureSensor();
window.humSensor = new phidget22.HumiditySensor();
window.moistSensor = new phidget22.VoltageRatioInput();
window.liteSensor = new phidget22.LightSensor();
window.pump = new phidget22.DigitalOutput();
pump.setHubPort(1);
pump.setIsHubPortDevice(true);

async function setup_sensors() {
    let errorCode = 0;
    try {await tempSensor.open(1000);} catch {errorCode |= 1;}    
    try {await humSensor.open(1000);} catch {errorCode |= 2;}
    try {await moistSensor.open(1000);} catch {errorCode |= 4;}
    try {await liteSensor.open(1000);} catch {errorCode |= 8;} 
    try {await pump.open(1000);} catch {errorCode |= 16;}
    return errorCode
}

setup_sensors()

In [ ]:
## We rest for ten seconds while the Phidgets try to connect.
sleep(10)

### Confirm the sensor connections

Run the following code to see if the five devices have all connected. All five lines should say "YES." If you don't have a sensor connected, this notebook will still work -- you just won't get data from that sensor. 

In [ ]:
%%js
element.text("Is the humidity sensor attached? " + (humSensor.attached ? "YES.":"NO."));
element.append("<br>Is the temperature sensor attached? " + (tempSensor.attached ? "YES.":"NO."));
element.append("<br>Is the moisture sensor attached? " + (moistSensor.attached ? "YES.":"NO."));
element.append("<br>Is the light sensor attached? " + (liteSensor.attached ? "YES.":"NO."))
element.append("<br>Is the pump attached? " + (pump.attached ? "YES.":"NO."));

## Step 3: Update data and connect the water pump

Here we define two functions. One to update the gauges with current data, and another to turn on the water pump for two seconds, then turns it off. This should be enough to give a small squirt of water to your plant.

These function is attached to the "Update Gauges" and "Click to Water" button, displayed below. Try pressing these button to see if they work. 

In [ ]:
## Some code to update the data

def update_gauges(button):
    display(Javascript("""
        if (tempSensor.attached) {
            IPython.notebook.kernel.execute(
                "gauges.data[0]['value'] = " + tempSensor.temperature); }
        if (humSensor.attached) {
        IPython.notebook.kernel.execute(
            "gauges.data[1]['value'] = " + humSensor.humidity); }
        if (moistSensor.attached) {
        IPython.notebook.kernel.execute(
            "gauges.data[2]['value'] = " + moistSensor.voltageRatio); }
        if (liteSensor.attached) {
        IPython.notebook.kernel.execute(
            "gauges.data[3]['value'] = " + liteSensor.illuminance); }
    """))
data_button.on_click(update_gauges)

def water_plant(widget):
    widget.button_style = 'info'
    display(Javascript("pump.setState(true);"))
    sleep(2)
    widget.button_style = 'success'
    display(Javascript("pump.setState(false);"))
water_button.on_click(water_plant)

display(dashboard)

## Step 4: Observing the results

Once you water the plant, you should see the moisture level for the plant change. 

The other data gauges are live as well. Can you see the temperature, humidity and light level changes throughout the day?

You will need to close this notebook and disconnect the Phidgets before see the live spreadsheet, which you need to reconnect the sensors.

## Final Step. Closing down the sensors

It is **really important** to close the sensor now, as otherwise they will keep busy forever, always trying to update the spreadsheet with the latest values. Also, if you just quit the notebook, the sensors may not disconnect properly, which will give you trouble the next time you try to connect.

So, don't skip this next step.

The following cell creates a button that you can click to close the Phidgets. Click it once you are all done with the Phidgets in this notebook. 

In [ ]:
from IPython.display import display, Javascript

def doDisconnect(b):
    display(Javascript("""
        (async () => {
            await humSensor.close();
            await tempSensor.close();
            await moistSensor.close();
            await liteSensor.close();
            await pump.close();
            usbconn.close();
            usbconn.delete();
            delete window.usbconn;
            element.text("You have disconnected the Phidgets.");
        })();
    """))

run_button = widgets.Button(
    description = 'IMPORTANT: Click to disconnect', 
        button_style='danger',layout=widgets.Layout(width='50%', height='80px')
)
print("Press this button when you are done, to disconnect the Phidgets")
run_button.on_click(doDisconnect)

display(run_button)


### Confirm

You can confirm the Phidgets are open or closed by running the following cell. 

If any device is still attached (true), try clicking the button above, again.

In [ ]:
%%js
element.text("Is the humidity sensor attached? " + (humSensor.attached ? "YES.":"NO."));
element.append("<br>Is the temperature sensor attached? " + (tempSensor.attached ? "YES.":"NO."));
element.append("<br>Is the moisture sensor attached? " + (moistSensor.attached ? "YES.":"NO."));
element.append("<br>Is the light sensor attached? " + (liteSensor.attached ? "YES.":"NO."))
element.append("<br>Is the pump attached? " + (pump.attached ? "YES.":"NO."));

## Conclusion

We have shown how to display Phidget sensor data from a plant and how to control the water pump for the plant. It uses a combination of Javascript and Python to the sensors and display the results. 

You may now go to the next notebook, which automates the watering of your plant. The computer will monitor the moisture of the soil every day and decide whether to pump in more water. The notebook is here: 
[plants-phidgets-auto-watering.ipynb](plants-phidgets-auto-watering.ipynb)

You might also like to go back to the data collection notebooks: 
- [plants-phidgets-data-google.ipynb](plants-phidgets-data-google.ipynb)
- [plants-phidgets-data-ethercalc.ipynb](plants-phidgets-data-ethercalc.ipynb)

[![Callysto.ca License](https://github.com/callysto/curriculum-notebooks/blob/master/callysto-notebook-banner-bottom.jpg?raw=true)](https://github.com/callysto/curriculum-notebooks/blob/master/LICENSE.md)